# Function bwlp

## Synopse

Low-Pass Butterworth frequency filter.

- **H = iabwlp(fsize, tc, n, option='circle')**

  - **H**: Image. 
   


  - **fsize**: Image. Filter size: a col vector: first element: rows, second: cols, etc. uses same convention as the return of size.
  - **tc**: Double. Cutoff period.
  - **n**: Double. Filter order.
  - **option**: String. Filter options. Possible values: 'circle' or 'square'.

In [ ]:
from numpy import *
import string

def iabwlp(fsize, tc, n, option='circle'):
    from iafftshift import iafftshift

    def test_exp(x, y):
        try:
            return x**(2*y)
        except:
            return 1E300 # Infinito!

    rows, cols = fsize[0], fsize[1]
    mh, mw = rows/2, cols/2
    rr, cc = meshgrid(arange(-mh,rows-mh), arange(-mw,cols-mw), indexing='ij') # center
    if string.find(string.upper(option), 'SQUARE') != -1:
        H = 1./(1.+(sqrt(2)-1)*(maximum(abs(1.*rr/rows) , abs(1.*cc/cols))*tc)**(2*n))
    else:
        aux1 = ravel(sqrt(((1.*rr)/rows)**2 + ((1.*cc)/cols)**2)*tc)
        aux2 = 0.*aux1 + n
        aux = reshape(map(test_exp, aux1, aux2), cc.shape)
        H = 1./(1+(sqrt(2)-1)*aux)
    H = iafftshift(H)
    return H

Description
===========

This function generates a frequency domain Low Pass Butterworth Filter with cutoff period ``tc`` and order ``n``. 
At the cutoff period the filter amplitude is about 0.7 of the amplitude at H(0,0). This function returns the mask filter with H(0,0). 
As the larger the filter order, sharper will be the amplitude transition at cutoff period. The minimum cutoff period is always 2 pixels, 
despite of the size of the frequency filter.

Examples
========


Example 1
---------

.. code:: python

    from ia636 import iabwlp, iadftview

    H2_10 = iabwlp([100,100],2,10) # cutoff period: 2 pixels, order: 10
    adshow(iadftview(H2_10))
    H4_1 = iabwlp([100,100],4,1) # cutoff period: 4, order: 1
    adshow(iadftview(H4_1))
    H8_100 = iabwlp([100,100],8,100) # cutoff period: 8, order: 100
    adshow(iadftview(H8_100))
    H4_1box = iabwlp([100,100],4,1,'square') # cutoff period: 4, order: 1
    adshow(iadftview(H4_1box))




Example 2
---------

.. code:: python

    f = adreadgray('cookies.pgm')
    adshow(f)
    F = fft.fft2(f)
    adshow(iadftview(F))
    H = iabwlp(F.shape,16,6)
    adshow(iadftview(H))
    G = F * H
    adshow(iadftview(G))
    g = fft.ifft2(G)
    g = abs(g).astype(uint8)
    adshow(g)






Equation
========


.. equation:: latex
    :align: left

    H(u,v) & = & \frac{1}{1+ (\sqrt{2}-1) (\sqrt{(\frac{u}{N})^2 + (\frac{v}{M})^2} \ \ t_c)^{2n}} \\ (u,v) & \in & \{ -\lfloor \frac{N}{2}\rfloor:N-\lfloor \frac{N}{2}\rfloor - 1 , -\lfloor \frac{M}{2} \rfloor:M-\lfloor \frac{M}{2}\rfloor-1 \} \\ t_c & \in & \{ 2 : \max\{N,M\} \}






.. code:: python testsuite
    :show_code:   no
    :show_output: no
    :show_images: no

    
    print 'testing iabwlp'
    print `floor(0.5 + 1E4*iabwlp([5,8],2,3)) / 1E4` == `array(
          [[ 0.9983,  0.9955,  0.9722,  0.8649,  0.6073,  0.8649,  0.9722,  0.9955],
           [ 0.9021,  0.8744,  0.774 ,  0.5813,  0.3537,  0.5813,  0.774 ,  0.8744],
           [ 0.9021,  0.8744,  0.774 ,  0.5813,  0.3537,  0.5813,  0.774 ,  0.8744],
           [ 0.9983,  0.9955,  0.9722,  0.8649,  0.6073,  0.8649,  0.9722,  0.9955],
           [ 1.    ,  0.9999,  0.9936,  0.9313,  0.7071,  0.9313,  0.9936,  0.9999]])`